In [1]:
from __future__ import division
import bokeh as bk
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')
import pandas as pd
import msgpack
import toolz
import scipy.stats as st
import matplotlib.pylab as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import blosc
import cPickle as pkl

In [2]:
def get_genes():
    with open('genes.msg', 'rb') as f:
        return set(msgpack.unpackb(f.read()))

def get_gene_map_intron():
    with open('gene_map_intron.msg', 'rb') as f:
        return dict(msgpack.unpackb(f.read()))

def read_gene(gene):
    df = pd.read_hdf('intron_events.h5', 'df', where="(gene == '%s')" % (gene,))
    df['nsuc'] = df['nread0']
    df['ntri'] = df['nread0'] + df['nread1']
    del df['nread0']
    del df['nread1']
    df = df.reset_index()
    del df['gene']
    del df['assay']
    df.name = gene
    return df

def read_gene_intron(gene_intron):
    gene, intron = gene_intron
    df = pd.read_hdf('intron_events.h5', 'df',
                     where="(gene == '%s') & (intron == %d)" % (gene, intron))
    df['nsuc'] = df['nread0']
    df['ntri'] = df['nread0'] + df['nread1']
    del df['nread0']
    del df['nread1']
    df = df.reset_index()
    del df['gene']
    del df['assay']
    df.name = gene
    return df

def load_entire_file():
    with open('intron_events.pkl.blp', 'rb') as f:
        return pkl.loads(blosc.decompress(f.read()))

def save_pickle():
    with open('intron_events.pkl.blp', 'wb') as f:
        f.write(blosc.compress(pkl.dumps(df, -1)))

In [3]:
%%time
genes = get_genes()
gene_map_intron = get_gene_map_intron()

CPU times: user 267 ms, sys: 79 ms, total: 346 ms
Wall time: 357 ms


Get all pair of gene-intron in a generator.

In [4]:
%%time
gi = gene_map_intron
gene_intron = ((k, v) for k in gi.keys() for v in gi[k])

CPU times: user 286 µs, sys: 0 ns, total: 286 µs
Wall time: 372 µs


In [5]:
%%time
df = load_entire_file()

CPU times: user 1.22 s, sys: 952 ms, total: 2.17 s
Wall time: 1.47 s


In [15]:
df.head()

assay  nread0  nread1
gene            intron                                      
ENSG00000000003 1       HG00096.1.M_111124_6       0       2
                1       HG00097.7.M_120219_2       0       2
                1       HG00099.1.M_120209_6       0       0
                1       HG00099.5.M_120131_3       0       2
                1       HG00100.2.M_111215_8       0       0

Load the entire DataFrame into memory

In [10]:
%%time
df = load_entire_file()

error: Error 211049427 : not a Blosc buffer or header info is corrupted

In [ ]:
blosc.compress

In [8]:
gene0 = toolz.first(genes)

In [14]:
%%time
dfv = df.loc[gene0]

CPU times: user 877 ms, sys: 1.71 s, total: 2.58 s
Wall time: 2.58 s


In [22]:
%%time
dfv = df.loc[gene0, 30]

CPU times: user 2.13 s, sys: 2.53 s, total: 4.66 s
Wall time: 4.66 s


In [23]:
dfv.head()

assay  nread0  nread1
gene            intron                                      
ENSG00000110514 30      HG00096.1.M_111124_6       6     107
                30      HG00097.7.M_120219_2       8     188
                30      HG00099.1.M_120209_6       3     142
                30      HG00099.5.M_120131_3       0     141
                30      HG00100.2.M_111215_8       4     118

Acceptance criterion

In [6]:
def accept(df):
    return np.percentile(df['ntri'].values, 90) < 1000

In [56]:
p = Pool(20)
ok = p.map(toolz.compose(accept, read_gene_intron), gene_intron)

IndexError: cannot do a non-empty take from an empty axes.

In [7]:
def does_it_return(gene_intron):
    try:
        gene, intron = gene_intron
        df = pd.read_hdf('intron_events.h5', 'df',
                         where="(gene == '%s') & (intron == %d)" % (gene, intron))
        df['nsuc'] = df['nread0']
        df['ntri'] = df['nread0'] + df['nread1']
        del df['nread0']
        del df['nread1']
        df = df.reset_index()
        del df['gene']
        del df['assay']
        df.name = gene
    except:
        return (gene, intron, False)
    return (gene, intron, True)

In [ ]:
pool = Pool(20)
results = []
for result in tqdm(pool.imap_unordered(does_it_return, gene_intron, chunksize=1000), mininterval=60, maxinterval=300):
    results.append(result)

7370001it [6:19:05, 322.66it/s]

In [62]:
cagado = [i for i in gene_intron if len(i) == 0 or len(i) == 1]

In [63]:
len(cagado)

0

In [10]:
df = read_gene(gene0)
df.head()

,intron,nsuc,ntri
0,1,1,3
1,1,0,3
2,1,0,4
3,1,0,3
4,1,0,2


In [27]:
p = Pool(20)
ok = p.map(toolz.compose(accept, read_gene), genes)

In [30]:
print("We have genes that have passed the %d" % sum(ok))

We have genes that have passed the 26721
